In [ ]:
import pandas as pd
from mantid.simpleapi import *
import matplotlib.pyplot as plt
import numpy as np
import lmfit as lm
import h5py
import os
import json
from datetime import datetime as dt
import pathlib
from mantid.geometry import ReflectionGenerator, ReflectionConditionFilter
from numpy.polynomial import Chebyshev as cheb

#This is the location of the SummaryTxt file
SummaryTxt = r'\\isis\inst$\NDXENGINX\Instrument\logs\journal\SUMMARY.TXT'

#This is the location of the files with the Collimator Groupings
GroupingFiles = r'C:\MantidInstall\scripts\Engineering\calib'

HDF files are a very useful and compact way to store 3D Data in a form that users can easily interact with. This cell creates the initial HDF file, as they can be very specific about naming, and after the data is filled in with other functions, a table is appended of all the parameters against all the runs. There is also a function for saving searches, peaks across banks, and orientation relations.

In [ ]:
def createHDF(name):
    fn = f'{name}.hdf5'
    check = pathlib.Path(fn)
    i = 1
    while check.exists():
        new_name = f'{fn[0:-5]}({i}).hdf5'
        check = pathlib.Path(new_name)
        i += 1
    return check

def saveSearch(name, search, df):
    currdir = os.getcwd()
    savedir = os.path.join(currdir, "Saves")
    newdir = os.path.join(savedir, str(name))
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    filedir = os.path.join(newdir,str(name))
    file = createHDF(filedir)
    dic = df.to_dict('list')
    with h5py.File(str(file),"w") as f:
        grp = f.create_group(search)
        for ColumnName in ['Inst','5DigitRunNumber','Users','Title','Date','Time','uAh','RB No.']:
            datset = f.create_dataset(f'{search}/{ColumnName}',data=dic[f'{ColumnName}'])
    f.close()
    #appendTableTwo(file)

def storeGrpPeaks(name, df, peaklen,hkls):
#def storeGrpPeaks(name,df,peaklen):
    currdir = os.getcwd()
    savedir = os.path.join(currdir, "Saves")
    newdir = os.path.join(savedir, str(name))
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    filedir = os.path.join(newdir,str(name))
    file = createHDF(filedir)
    dic = df.to_dict('list')
    #print(dic)
    with h5py.File(str(file),"w") as f:
        #for i in range(peaklen):
        for i in range(len(hkls)):
            peakPara = []
            grp = f.create_group(f'{hkls[i]}-Peak')
            for ColumnName in ['Height', 'Height_Err', 'PeakCentre', 'PeakCentre_Err', 'Sigma', 'Sigma_Err', 'Integrated Intensity', 'Integrated Intensity_Err', 'Chi_squared']:
                #datset = f.create_dataset(f'Peak No.{k}/{ColumnName}', data=dic[f'Peak No.{k}-{ColumnName}'])
                try:
                    datset = f.create_dataset(f'{hkls[i]}-Peak/{ColumnName}', data=dic[f'{hkls[i]}-{ColumnName}'])
                except KeyError:
                    print(f"Can't save {hkls[i]}-{ColumnName} into HDF. This may be due to an error with the Peak Detection.")
                    pass
    f.close()
    try:
        appendTable(file)
    except IndexError:
        pass

def saveOrientation(name,corr,table,peaklen,orderDic,peaks):
    currdir = os.getcwd()
    savedir = os.path.join(currdir, "Saves")
    newdir = os.path.join(savedir, str(name))
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    filedir = os.path.join(newdir,f'{name}_Orientation_{corr}_Correlation')
    file = createHDF(filedir)
    with h5py.File(str(file),"w") as f:
        grp = f.create_group(f'{name}-Correlations')
        i = 0
        for peak in peaks:
            #grp = f.create_group(f'Peak No.{k}')
            peakCorr = []
            for j in range(peaklen):
                peakCorr.append(table[i,j])
            #print(peakCorr)
            datset = f.create_dataset(f'{name}-Correlations/{peak}', data=peakCorr)
            i = i + 1
        tableset = f.create_dataset(f'{name}-Correlations/Full Table', shape=(peaklen,peaklen), maxshape=(None,None), data=table)
    f.close()
    orderedPeaksHDF(file,orderDic,peaks)

def appendTable(filename):
    with h5py.File(filename, "r") as f:
        keyList = list(f.keys())
        #print(keyList)
        rows = len(keyList)
        cols = 0
        for i in range(rows):
            check = f[keyList[i]]
            #print(check)
            if keyList[i] == "Tables":
                i = i + 1
            else:
                checkList = list(check)
                params = len(checkList)
                colTest = len(list(check[checkList[0]]))
                #colTest = len(list(check["Inst"]))
                if colTest > cols:
                    cols = colTest
                else:
                    cols = cols
    f.close()
    with h5py.File(filename, "a") as f:
        for k in range(params):
            path = "Tables/" + checkList[k]
            tableData = f.create_dataset(path, (rows,cols), maxshape=(None,None), dtype='float')
            for i in range(rows):
                run = f[keyList[i]]
                dat = list(run[checkList[k]])
                diff = cols - len(dat)
                for j in range(diff):
                    dat.append(0)
                tableData[i,:] = dat
            
    f.close()

#This is to create a HDF of the ordered peaks
def orderedPeaksHDF(filename, orderDic,peaks):
    with h5py.File(filename,"a") as f:
        for peak in peaks:
            path = f'Ordered_Peaks/{peak}_ordered'
            arr = orderDic[f'{peak}_ordered']
            datset = f.create_dataset(path, data=arr)
    f.close()
        
    
#There's an issue with the types when making a table
def appendTableTwo(filename):
    with h5py.File(filename, "r") as f:
        keyList = list(f.keys())
        #print(keyList)
        rows = len(keyList)
        cols = 0
        for i in range(rows):
            check = f[keyList[i]]
            print(check)
            if keyList[i] == "Tables":
                i = i + 1
            else:
                checkList = list(check)
                params = len(checkList)
                colTest = len(list(check[checkList[0]]))
                #colTest = len(list(check["Inst"]))
                if colTest > cols:
                    cols = colTest
                else:
                    cols = cols
    f.close()
    with h5py.File(filename, "a") as f:
        ds_dtype = []
        for i in range(params):
            checkType = type(checkList[i][0])
            ds_dtype.append((f'{checkList[i]}',checkType))
        #print(ds_dtype)
        ds_arr = np.recarray((cols,),dtype=ds_dtype)
        for i in range(params):
            for j in range(rows):
                run = f[keyList[j]]
                dat = list(run[checkList[i]])
                ds_arr[f'{checkList[i]}'] = np.asarray(dat)
        dset = f.create_dataset('Table', dtype=ds_dtype, shape=(cols,), maxshape=(None) )
        for i in range(params):
            dset[f'{checkList[i]}',0:i] = np.asarray(list(run[checkList[i]]))
            dset[f'{checkList[i]}',0:i] = np.asarray(list(run[checkList[i]]))
            dset[f'{checkList[i]}',0:i] = np.asarray(list(run[checkList[i]]))
            dset[f'{checkList[i]}',0:i] = np.asarray(list(run[checkList[i]]))
            
    f.close()

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

#Borrow some of this code to improve filesearch
def LoadSummaryTxt(Filename,col,search,sort):
    col_names = ['Inst','5DigitRunNumber','Users','Title','Date','Time','uAh','RB No.']
    col_width = [(0,3),(3,8),(8,28),(28,52),(52,63),(63,72),(72,80),(80,88)]
    # Run 53079 on Engin-X has corrupted row
    InvalidRows = [24493,41604,41605,41606,41607,41608]
    datas = pd.read_fwf(Filename, names=col_names, colspecs=col_width, na_filter=False,  encoding='ANSI', skiprows=InvalidRows) # encoding='ISO-8859-1'
    #print(datas)
    df = pd.DataFrame(datas)
    RunNumberDiff = df['5DigitRunNumber'].diff()
    RunNumberDiff = RunNumberDiff.replace(-99999, 1).replace(-99998, 2) # Replace wraparound values
    RunNumberDiff.iloc[0] = df['5DigitRunNumber'].iloc[0]
    df['RunNumber'] = RunNumberDiff.cumsum().astype('int64')
    df['uAh'] = pd.to_numeric(df['uAh'], errors='coerce')
    df.set_index('RunNumber', inplace=True)
    try:
        searchResults = df[df[str(col)].str.contains(f'{search}')].sort_values(str(sort))
    except AttributeError:
        searchResults = df[str(col)].dtype
    searchQuery = f'{col}-{search}'
    hdfName = f'{search}_sort_by_{sort}'
    #saveSearch(hdfName,searchQuery,searchResults)
    #return df
    return searchResults

df = LoadSummaryTxt(SummaryTxt,"Date","2024","RunNumber")
df
#pd.to_numeric(df['RunNumber']).diff().sort_values()
#df.plot.hist()
#"scan" in df.Title
#df
#df[df["Title"].str.contains("CeO2")].sort_values('uAh')   #... ['uAh'].hist(bins=100)
#df.to_dict('list')

In [ ]:
def fileSearch(run,exp,bank):
    exps = ['Cryo','Furnace','Pressure','X Scan','Y Scan','Z Scan']
    if len(str(run)) < 5:
        zero_diff = 5 - len(str(run))
        for i in range(zero_diff):
            run = f'0{run}'
            
    yr = dt.now().strftime('%y')
    cy = int(yr) + 1

    if bank == '1':
        sMin_One = 0
        sMax_One = 1200
        sMin_Two = 1
        sMax_Two = 1199
    elif bank == '2':
        sMin_One = 1201
        sMax_One = 2400
        sMin_Two = 1200
        sMax_Two = 2399
    elif bank == 'Both':
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    else:
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    for i in reversed(range(2,int(cy))):
        if i < 10:
            year = f'0{i}'
        else:
            year = f'{i}'
        if i < 17:
            apps = [['cryo_1','cryo_temp1'],'furnace','position','x','y','z']
            if exp == exps[0]:
                if i < 10:
                    app = apps[0][0]
                else:
                    app = apps[0][1]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        else:
            apps = ['Temp_1','Temp_3','position','X_position','Y_position','Z_position']
            if exp == exps[0]:
                app = apps[0]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        for cycle in [5,4,3,2,1]:
            if i < 8:
                name_one = f'ENG{run}'
                name_two = f'ENG{run}'
            else:
                name_one = f'ENG00{run}'
                name_two = f'ENGINX00{run}'
            ext_one = '.raw'
            ext_two = '.nxs'
            testdir_one = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_one}'
            testdir_two = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_one}'
            testdir_three = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_two}'
            testdir_four = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_two}'
            check_one = pathlib.Path(testdir_one)
            check_two = pathlib.Path(testdir_two)
            check_three = pathlib.Path(testdir_three)
            check_four = pathlib.Path(testdir_four)
            if check_one.exists():
                dire = testdir_one
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_two.exists():
                dire = testdir_two
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_three.exists():
                dire = testdir_three
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_four.exists():
                dire = testdir_four
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            else:
                pass

In [ ]:
PaBrun = 324889
#Testing the code with Niobium Runs
#PaBrun = 351364
#PaBrun = 351464
#PaBrun = 351538
#PaBrun = 351637

In [ ]:
import lmfit as lm

def gaussian_function(x, a, x0, s,b):
    return b+a*np.exp(-(x-x0)**2/(2*s**2))
   
def residualGauss(pars,x,data):
    model = gaussian_function(x, a=pars['a'],x0=pars['x0'],s=pars['s'],b=pars['b']) 
    return model - data

In [ ]:
#These are all the dependencies for the Cubic Peak Fit. Other functions that had to be brought in to make it work
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def find_nearest_ind(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def logData(x,y):
    yNorm = y/np.amax(y)
    xLog = np.log(x)
    inter = (x[-1] - x[0])/len(x)
    xSize = np.arange(xLog[0],xLog[-1],inter)
    xLogInterp = np.interp(xSize,x,xLog)
    yLogInterp = np.interp(xSize,xLog,yNorm)
    return xSize,yLogInterp,inter

def logInter(run,datx,bins):    
    _,temp,sumSpec = fileSearch(run,exp=None,bank=None)
    y = mtd[sumSpec].dataY(0)
    x = mtd[sumSpec].dataX(0)[0:len(y)]
    if x[0] < datx[0]:
        x_datx_ind = find_nearest_ind(x, datx[0])
#        print(x_datx_ind)
        x = x[x_datx_ind:list(x).index(x[-1])]
        y = y[x_datx_ind:len(y)]
    if len(y) < len(datx):
        diff_x = int(abs(len(datx) - len(x))/2)
        diff_y = int(abs(len(datx) - len(y))/2)
        y = np.pad(y, (diff_y,diff_y), 'constant')
        x = np.pad(x, (diff_x,diff_x), 'linear_ramp',end_values=(datx[0],datx[-1]))
    if len(x) < len(y):
        dist = len(y) - len(x)
        inval = x[-1]/len(datx)
        x = np.pad(x, (0,dist), 'linear_ramp',end_values=(datx[0],(datx[-1]+(dist*inval))))
    if len(x) > len(y):
        dist = len(x) - len(y)
        y = np.pad(y, (0,dist), 'constant')
#    print(len(datx),len(x),len(y))
    yNorm = y/np.amax(y)
    xLog = np.log(x)
    xSize = np.arange(xLog[0],xLog[-1],bins)
#    print(len(xSize),len(xLog),len(yNorm))
    xLogInterp = np.interp(xSize,x,xLog)
    yLogInterp = np.interp(xSize,xLog,yNorm)
    return x,y,xSize,yLogInterp,temp

def noise_gen(level,size):
    noi_arr = np.random.normal(0,level,size)
    #print(noi_arr)
    for i in range(len(noi_arr)):
        if noi_arr[i] < 0:
            noi_arr[i] = 0
    return noi_arr
    
def CIFData(element,noise_lvl,size,drange=[0.67,3.36],plot_CIF=True):
    #This is to create x values for the data
    x = np.linspace(drange[0],drange[1],size)
    plot = np.zeros(len(x),)
    #The bank distance and source distance are from the ENGIN-X manual in order to replicate, as much as possible,
    #realistic data
    ws = CreateSampleWorkspace(Function='Multiple Peaks',BankDistanceFromSample=0.01,SourceDistanceFromSample=50)
    LoadCIF(ws,f'C:/Users/ynn46697/Documents/ISIS/Grain_Analysis/Periodic_Table_CIFs/{element}.cif')
    sample = ws.sample().getCrystalStructure()
    unitCell = sample.getUnitCell()
    a = unitCell.a()
    pg = sample.getSpaceGroup().getPointGroup()
    generator = ReflectionGenerator(sample)
    hkls = list(generator.getUniqueHKLs(drange[0],drange[-1]))
    if plot_CIF == False:
        #Certain functions only need the CIF Data generation as a shortcut to determining hkls
        return hkls
    else:
        pass
    peaks = list(generator.getDValues(hkls))
    Fsq = list(generator.getFsSquared(hkls))
    j = list(map(lambda x : len(pg.getEquivalents(x)), hkls))
    s = list(map(lambda x : x*0.005, peaks))
    p = []
    fl = []
    height = []
    for i in range(len(peaks)):
        p.append((j[i]*Fsq[i]*(peaks[i]**4))/(a**3)) #This equation comes from the Introduction to 
        #the Characterization of Residual Stress by Neutron Diffraction book by M.T. Hutchings, P.J. Withers, et al.
        #with some parameters missing as these are undeterminable or negligible without a real experiment
        fl.append(flux[find_nearest_ind(flux_pt,peaks[i])])
        height.append(fl[i] * p[i])
    for peak in peaks:
        height_val = height[peaks.index(peak)]
        s_val = s[peaks.index(peak)]
        b = 0
        plot += gaussian_function(x,height_val,peak,s_val,b)
    noise = noise_gen((np.amax(plot)*noise_lvl),len(plot))
    plot = plot + noise
    return x,plot,hkls

def bg_fit(x,y,loop=25):
    #This uses chebyshev polynomials to fit to the background
    cheb_fit,_ = cheb.fit(x,y,0,full=True)
    coef = np.sum(cheb_fit.coef)
    for i in range(1,loop):
        cheb_fit,params = cheb.fit(x,y,i,full=True)
        new_coef = np.sum(cheb_fit.coef)
        coef_check = abs(0 - coef)
        new_check = abs(0 - new_coef)
        if new_check < coef_check:
            coef = new_coef
            best_fit = cheb_fit
        else:
            pass
    bg_x,bg_y = best_fit.linspace(n=len(x)) #This can be used to add a background fit to peak fits
    y_hat = abs(y - bg_y) #This can be used to remove the background from particularly noisy data
    return bg_x,bg_y,y_hat

_,_,sumSpec = fileSearch(358221,exp=None,bank=None)

flux = mtd[sumSpec].dataY(0)
flux_pt = mtd[sumSpec].dataX(0)[0:len(flux)]

In [ ]:
#This fits peaks for fcc and bcc
def cubicPeakFit(run,exp,bank,size,bg,peak_bg):
    paramsArr = []
    #First it generates a Vanadium and Copper sample then logs their x values
    bcc_x,bcc_y,bcc_hkls = CIFData('V',0.005,size)
    fcc_x,fcc_y,fcc_hkls = CIFData('Cu',0.005,size)
    bcc_xS,bcc_yLog,bcc_inter = logData(bcc_x,bcc_y)
    fcc_xS,fcc_yLog,fcc_inter = logData(fcc_x,fcc_y)
    #After this, it retrieves the run and logs the x values of its data too
    x,y,dat_xS,dat_yLog,temp = logInter(run,bcc_x,max(bcc_inter,fcc_inter))
    bg_x,bg_y,y_hat = bg_fit(x,y,loop=25)

    #This determines the furthest peak in the dataset
    lim = 1
    dist = 0.025
    rep = int(lim / dist)
    for i in range(rep):
        dat_end = int(len(x)*lim)
        dat_break = int(len(x)*(lim-dist))
        dat_trunx = x[dat_break:dat_end]
        dat_trunc = y[dat_break:dat_end]
        dat_peaks,dat_params = fp(dat_trunc,prominence=15,distance=100)
        lim = lim - dist
        if len(dat_peaks) > 0:
            break
    dat_peaks,_ = fp(dat_trunc)
    dat_yPeaks = list(dat_trunc[dat_peaks])
    dat_x0 = list(dat_trunx[dat_peaks])
    dat_far = np.amax(dat_yPeaks) #This is the furthest peak in y
    dat_fax = dat_x0[dat_yPeaks.index(dat_far)] #And then in x
    #The data is then correlated against both the bcc and fcc data
    corr_bcc = np.correlate(bcc_yLog,dat_yLog,mode='full')
    corr_fcc = np.correlate(fcc_yLog,dat_yLog,mode='full')
    #Then the peaks of the correlation are identified
    corr_axes = np.linspace(0,len(corr_bcc),len(corr_bcc))
    corr_bcc_peaks,corr_bcc_params = fp(corr_bcc,prominence=1)
    corr_fcc_peaks,corr_fcc_params = fp(corr_fcc,prominence=1)
    #These peaks are then collated
    bcc_x = corr_axes[corr_bcc_peaks]
    bcc_y = corr_bcc[corr_bcc_peaks]
    fcc_x = corr_axes[corr_fcc_peaks]
    fcc_y = corr_fcc[corr_fcc_peaks]
    #And sorted
    bcc_sort = sorted(bcc_y)
    fcc_sort = sorted(fcc_y)
    #Then the difference between the two highest peaks within the prominence are determined
    bcc_peak_diff = bcc_sort[-1]-bcc_sort[-2]
    fcc_peak_diff = fcc_sort[-1]-fcc_sort[-2]

    #The second peak of each crystal structure are then estimated in case the correlation failed
    a_bcc = dat_fax*np.sqrt(2)
    a_fcc = dat_fax*np.sqrt(3)
    d_two_bcc = a_bcc / 2
    d_two_fcc = a_fcc / 2
    loc_two_bcc = find_nearest_ind(x,d_two_bcc)
    loc_two_fcc = find_nearest_ind(x,d_two_fcc)
    range_two_bcc = range(loc_two_bcc-200,loc_two_bcc+200)
    range_two_fcc = range(loc_two_fcc-200,loc_two_fcc+200)
    bcc_two_peak,bcc_two_par = fp(y[range_two_bcc],prominence=15)
    fcc_two_peak,fcc_two_par = fp(y[range_two_fcc],prominence=15)
    
    if bcc_peak_diff > fcc_peak_diff:
        #This is a double check because sometimes the dataset reads as the wrong crystal structure
        if y[loc_two_fcc] > y[loc_two_bcc]:
            #This is a triple check
            if len(bcc_two_par['prominences']) == len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            elif len(bcc_two_par['prominences']) > len(fcc_two_par['prominences']):
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
            elif len(bcc_two_par['prominences']) < len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            else:
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
        else:
            if len(bcc_two_par['prominences']) == len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            elif len(bcc_two_par['prominences']) > len(fcc_two_par['prominences']):
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
            elif len(bcc_two_par['prominences']) < len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            else:
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
    else:
        if y[loc_two_bcc] > y[loc_two_fcc]:
            if len(bcc_two_par['prominences']) == len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            elif len(bcc_two_par['prominences']) > len(fcc_two_par['prominences']):
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
            elif len(bcc_two_par['prominences']) < len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            else:
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
        else:
            if len(bcc_two_par['prominences']) == len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            elif len(bcc_two_par['prominences']) > len(fcc_two_par['prominences']):
                cryst = 'bcc'
                hkls = bcc_hkls
                a = a_bcc
            elif len(bcc_two_par['prominences']) < len(fcc_two_par['prominences']):
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
            else:
                cryst = 'fcc'
                hkls = fcc_hkls
                a = a_fcc
    #This was for testing, to make sure it's reading the correct peak, crystal structure, and lattice parameter            
    check_arr = [dat_fax,cryst,a]

    x0 = []
    x0_idx = []
    hkl_arr = []
    #print(hkls)
    for hkl in hkls:
        h = hkl[0]
        k = hkl[1]
        l = hkl[2]
        #For each hkl, this calculates a peak then adds it to an array
        peak = a / np.sqrt(h**2+k**2+l**2)
        x0_ind = find_nearest_ind(x, peak)
        #print(peak,x0_ind)
        #print(find_nearest(x,peak))
        if x0_ind not in x0_idx:
            hkl_arr.append([f'{int(h)}{int(k)}{int(l)}'])
            x0.append(peak)
            x0_idx.append(x0_ind)
        else:
            pass
    plot = np.zeros(len(x),)
    for peak in x0:
        #This array is then used to perform the peak fitting as before
        peak_hkl = hkl_arr[x0.index(peak)]
        height = y[find_nearest_ind(x, peak)]
        #Due to the nature of these peaks being calculated not found, this finds the nearest approximation within
        #the data
        xOne = x[find_nearest_ind(x, peak)-75:find_nearest_ind(x, peak)]
        xTwo = x[find_nearest_ind(x, peak)+1:find_nearest_ind(x, peak)+75]
        yOne = y[find_nearest_ind(x, peak)-75:find_nearest_ind(x, peak)]
        yTwo = y[find_nearest_ind(x, peak)+1:find_nearest_ind(x, peak)+75]
        xOneArr = xOne[list(np.argwhere(yOne <= height/2))]
        xTwoArr = xTwo[list(np.argwhere(yTwo <= height/2))]
        try:
            x1 = np.amax(xOneArr)
            x2 = np.amin(xTwoArr)
            fwhm = x2 - x1
            sigma = fwhm / (2 * m.sqrt(2 * m.log(2)))
        except ValueError:
            #Failsafe if the code can't calculate fwhm
            sigma = 0.005 * peak
        if peak_bg == True:
            backg = bg_y
        else:
            backg = 0
        peak_ws = CreateSampleWorkspace()
        #Rather than lmfit, this function uses PlotPeakByLogValue as this function is much more rigid with the x0 values
        PeakFitString = f"name=Gaussian,Height={height},PeakCentre={peak},Sigma={sigma}"
        PlotPeakByLogValue(Input=peak_ws, OutputWorkspace='PeakFit', Function=PeakFitString, FitType='Individual')
        fit_res = mtd['PeakFit'].toDict()
        height_fit = fit_res['Height'][0]
        peak_fit = fit_res['PeakCentre'][0]
        sig_fit = fit_res['Sigma'][0]
        ii_fit = fit_res['Integrated Intensity'][0]
        para = [temp,peak_hkl,peak,height_fit,sig_fit,bg_y[find_nearest_ind(x, peak)],ii_fit]
        paramsArr.append(para)
        best_fit = gaussian_function(x=x, a=height_fit, x0=peak_fit, s=sig_fit,b=backg)
        if x0.index(peak) > 0:
            for j in range(len(plot)):
                check = [best_fit[j],plot[j]]
                bestCheck = np.amax(check)
                plot[j] = bestCheck
        else:
            plot = best_fit
    if bg == True:
        y = y_hat
    else:
        pass
    #print(x0_idx)
    return temp, x, y, plot, paramsArr

#This takes a list of peak locations provided by the user and fits peaks to it
def inputPeakFit(run,exp,bank,x0,cryst,size,bg,peak_bg):
    x0.sort()
    filepath, temp, sumSpec = fileSearch(run,exp,bank)
    y = mtd[sumSpec].dataY(0)
    x = mtd[sumSpec].dataX(0)[0:len(y)]
    bg_x,bg_y,y_hat = bg_fit(x,y,loop=25)
    x = list(x)
    x0_ind = []
    yPeak = []
    plot = []
    for i in range(len(x0)):
        #This makes sure the x0 values are in the x data
        x0_new = find_nearest(x, x0[i])
        x0[i] = x0_new
        x0_ind.append(x.index(x0[i]))
    for ind in x0_ind:
        #This searches the range around x0 in the y axis to make sure the x0 determined fits
        lb = ind - 100
        ub = ind + 100
        x0_range = list(range(lb,ub))
        yPeak_range = y[x0_range]
        yPeak.append(np.amax(yPeak_range))
    y = list(y)
    for i in range(len(yPeak)):
        #This matches the x0 back to the y
        peak_ind = y.index(yPeak[i])
        x0[i] = x[peak_ind]
        
    x0_rat = x0[-1]/x0[-2]
    if cryst == None:
        #If the user doesn't provide a crystal structure, this uses the ratios of the first and second peak's
        #location to determine which structure it may be
        cryst_arr = ['scc','bcc','fcc','fcc_dia']
        scc_rat = np.sqrt(2)/np.sqrt(1)
        bcc_rat = np.sqrt(4)/np.sqrt(2)
        fcc_rat = np.sqrt(4)/np.sqrt(3)
        fccdia_rat = np.sqrt(8)/np.sqrt(3)
        rats_arr = [abs(x0_rat-scc_rat),abs(x0_rat-bcc_rat),abs(x0_rat-fcc_rat),abs(x0_rat-fccdia_rat)]
        rats_best = np.amin(rats_arr)
        cryst = cryst_arr[rats_arr.index(rats_best)]
    if cryst == 'fcc':
        hkls = CIFData('Cu',0.005,size,drange=[0.01,x[-1]],plot_CIF=False)
    elif cryst == 'bcc':
        hkls = CIFData('V',0.005,size,drange=[0.01,x[-1]],plot_CIF=False)
    elif cryst == 'scc':
        hkls = CIFData('Po',0.005,size,drange=[0.01,x[-1]],plot_CIF=False)
    elif cryst == 'fcc_dia':
        hkls = CIFData('Sn',0.005,size,drange=[0.01,x[-1]],plot_CIF=False)
    else:
        hkls = CIFData('Po',0.005,size,drange=[0.01,x[-1]],plot_CIF=False)
    paramsArr = []
    x0_check = []
    for peak in x0:
        x0_idx = x.index(peak)
        #This is to prevent nearby peaks being repeated as the same hkl
        if x0_idx in x0_check:
            pass
        else:
            x0_check.append(x0_idx)
            try:
                peak_hkl = hkls[x0.index(peak)]
            except IndexError:
                #If the CIF Data can't find a hkl for the selected peak, this allows the code to continue
                peak_hkl = [0,0,0]
            height = y[x0_idx]
            xOne = x[x0_idx-75:x0_idx]
            xTwo = x[x0_idx+1:x0_idx+75]
            yOne = y[x0_idx-75:x0_idx]
            yTwo = y[x0_idx+1:x0_idx+75]
            xOne = np.array(xOne)
            xTwo = np.array(xTwo)
            xOneArr = xOne[list(np.argwhere(yOne <= height/2))]
            xTwoArr = xTwo[list(np.argwhere(yTwo <= height/2))]
            try:
                x1 = np.amax(xOneArr)
                x2 = np.amin(xTwoArr)
                fwhm = x2 - x1
                sigma = fwhm / (2 * m.sqrt(2 * m.log(2)))
            except ValueError:
                sigma = 0.05 * peak
            if peak_bg == True:
                backg = bg_y
            else:
                backg = 0
            peak_ws = CreateSampleWorkspace()
            #Again using PlotByPeakLogValue as the user already knows where the x0 is
            PeakFitString = f"name=Gaussian,Height={height},PeakCentre={peak},Sigma={sigma}"
            PlotPeakByLogValue(Input=peak_ws, OutputWorkspace='PeakFit', Function=PeakFitString, FitType='Individual')
            fit_res = mtd['PeakFit'].toDict()
            height_fit = fit_res['Height'][0]
            peak_fit = fit_res['PeakCentre'][0]
            sig_fit = fit_res['Sigma'][0]
            ii_fit = fit_res['Integrated Intensity'][0]
            para = [temp,peak_hkl,peak,height_fit,sig_fit,bg_y[find_nearest_ind(x, peak)],ii_fit]
            paramsArr.append(para)
            x = np.array(x)
            best_fit = gaussian_function(x=x, a=height_fit, x0=peak_fit, s=sig_fit,b=backg)
            if x0.index(peak) == 0:
                plot = best_fit
            else:
                for i in range(len(plot)):
                    check = np.amax([plot[i],best_fit[i]])
                    plot[i] = check
            x = list(x)
        if bg == True:
            y = y_hat
        else:
            pass
    print(x0_check)
    return temp, x, y, plot, paramsArr

In [ ]:
#PeakAcrossBanks with the peaks found by Scipy.signal and the fit done by mantid
#This is the PaB cell that runs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.signal import find_peaks as fp
import math as m

import pandas as pd
from itables import init_notebook_mode, options
init_notebook_mode(all_interactive=True)
options.maxBytes = 0

#from ipyaggrid import Grid

def PeakAcrossBanks(RunNo,peak_list=None,PixelGrouping=None,RebinParams='-0.0005'):
    # Load and focus data
    #filepath = fileSearch(str(RunNo))
    filepath,_,sumSpec = fileSearch(run=RunNo,exp=None,bank=None)
    testX = mtd[sumSpec].dataX(0)
    DiffnDataWS = Load(Filename=filepath, OutputWorkspace='DiffnDataWS', SpectrumMax=2400)
    ConvertUnits(InputWorkspace='DiffnDataWS', OutputWorkspace='DiffnDataWS-dSp', Target='dSpacing', AlignBins=True)
    # Group pixels and d-spacing axis to appropriate bin sizes
    LoadDetectorsGroupingFile(InputFile=PixelGrouping, InputWorkspace='DiffnDataWS-dSp', OutputWorkspace='XML-SpatialGrouping')
    GroupDetectors(InputWorkspace='DiffnDataWS-dSp', OutputWorkspace='DiffnDataWS-dSp-grp', IgnoreGroupNumber=False, CopyGroupingFromWorkspace='XML-SpatialGrouping')
    Rebin(InputWorkspace='DiffnDataWS-dSp-grp', OutputWorkspace='DiffnDataWS-dSp-grp-reb', Params=RebinParams)
    # Fit peak(s) to all spectra in workspace
    NumSpectra = mtd['DiffnDataWS-dSp-grp-reb'].getNumberHistograms()
    SpectraRefString = ";".join([f'DiffnDataWS-dSp-grp-reb,sp{i+1}' for i in range(NumSpectra)])
    #peaklen, peaklist, x0, x, y = peakFit(RunNo)
    #hkls,a,X0,struc = HKL_Calc(x0)
    if peak_list == None:
        _, x, y, plot, paramsArr = cubicPeakFit(run=RunNo,exp=None,bank=None,size=len(testX),bg=False,peak_bg=True)
    else:
        _, x, y, plot, paramsArr = inputPeakFit(run=RunNo,exp=None,bank=None,x0=peak_list,cryst=None,size=len(testX),bg=False,peak_bg=True)
    x0 = []
    hkls = []
    for para in paramsArr:
        x0.append(para[2])
        hkls.append(para[1])
    #print(hkls)
    #print(x0)
    peaklen = len(x0)
    #print(peaklen)
    #print(hkls)
    dMin, dMax = np.amin(x), np.amax(x)
    FitResults=None # Merge
    for i in range(peaklen):
        PeakPosition = x0[i]
        #print(PeakPosition)
        if dMin < PeakPosition < dMax:
            PeakHeights=[]
            for j in range(NumSpectra):
                PeakHeights.append(mtd['DiffnDataWS-dSp-grp-reb'].dataY(j)[mtd['DiffnDataWS-dSp-grp-reb'].yIndexOfX(PeakPosition)])
            PeakHeight = np.mean(PeakHeights)
            #print(f'Height = {PeakHeight}')
            #Add width
            #width = findSigma(PeakPosition,x,y)
            #PeakFitString = f"name=Gaussian,Height={PeakHeight},PeakCentre={PeakPosition},Sigma={width}"
            #Calculating sigma seemed to create a lot of NaNs for some reason
            PeakFitString = f"name=Gaussian,Height={PeakHeight},PeakCentre={PeakPosition},Sigma={PeakPosition*0.005}"
            PlotPeakByLogValue(Input=SpectraRefString, OutputWorkspace='DiffnDataWS-dSp-grp-reb-Fit', Function=PeakFitString, FitType='Individual', 
                               StartX=PeakPosition*0.98, EndX=PeakPosition*1.02)
            # Convert results from mantid TableWorkspace to Pandas, and add extra info
            ResultsDict = mtd['DiffnDataWS-dSp-grp-reb-Fit'].toDict()
            #print(ResultsDict)
            ResultsDict['GroupNo'] = ResultsDict.pop('axis-1')
            # Rename column headings to include peak name - could use MultiIndex for this?
            for ColumnName in ['Height', 'Height_Err', 'PeakCentre', 'PeakCentre_Err', 'Sigma', 'Sigma_Err', 'Integrated Intensity', 'Integrated Intensity_Err', 'Chi_squared']:
                k = i + 1
                #ResultsDict[f'Peak No.{k}-{ColumnName}'] = ResultsDict.pop(ColumnName)
                ResultsDict[f'{hkls[i]}-{ColumnName}'] = ResultsDict.pop(ColumnName)
            ResultsDF = pd.DataFrame.from_dict(ResultsDict).set_index("GroupNo")
            
            if FitResults is not None:
                FitResults = pd.merge(FitResults, ResultsDF, on="GroupNo") # , index=ResultsDict['GroupNo']
            else:
                FitResults = ResultsDF
    #print(ResultsDict.keys())
    dfHDF = FitResults
    #print(type(FitResults))
    dfHDF.reset_index(drop=True)

    #print(FitResults.keys())
    name = f'{RunNo}_Grp_HDF'
    #print(FitResults.keys())
    try:
        storeGrpPeaks(name, FitResults, peaklen,hkls)
    except ValueError:
        pass
    #storeGrpPeaks(name,FitResults,peaklen)

    Azimuths = [np.rad2deg(mtd['DiffnDataWS-dSp-grp-reb'].spectrumInfo().azimuthal(i)) for i in range(NumSpectra)]
    TwoThetas = [np.rad2deg(mtd['DiffnDataWS-dSp-grp-reb'].spectrumInfo().signedTwoTheta(i)) for i in range(NumSpectra)]
    #Angles= [np.rad2deg(mtd['DiffnDataWS-dSp-grp-reb'].spectrumInfo().geographicalAngles(i)) for i in range(NumSpectra)]
    FitResults['Azimuth']=Azimuths
    FitResults['TwoTheta']=TwoThetas
    #FitResults['Angles']=Angles
    for PixelGroup in FitResults.index:
        # Fix issues with signs and degenerate values in the angle data
        if -90 < FitResults.loc[PixelGroup, "Azimuth"] < 90:   # Indicates north bank
            FitResults.loc[PixelGroup, "TwoTheta"] = abs(FitResults.loc[PixelGroup, "TwoTheta"])
        if FitResults.loc[PixelGroup, "Azimuth"] < -90:        # -163 or -171.5, corresponds to -17 or -8.5
            FitResults.loc[PixelGroup, "Azimuth"] = -180 - FitResults.loc[PixelGroup, "Azimuth"]
            FitResults.loc[PixelGroup, "TwoTheta"] = -1 * abs(FitResults.loc[PixelGroup, "TwoTheta"])
        if FitResults.loc[PixelGroup, "Azimuth"] > 90:        # 163 or 171.5, corresponds to 17 or 8.5
            FitResults.loc[PixelGroup, "Azimuth"] = 180 - FitResults.loc[PixelGroup, "Azimuth"]
            FitResults.loc[PixelGroup, "TwoTheta"] = -1 * abs(FitResults.loc[PixelGroup, "TwoTheta"])
        if abs(FitResults.loc[PixelGroup, "TwoTheta"]) < 10:  # Degenerate case gives erroneous values - form of gimbal lock?
            FitResults.loc[PixelGroup, "Azimuth"] = 0
            FitResults.loc[PixelGroup, "TwoTheta"] = -90
            
    return FitResults, name
    
def PlotPeaksAcrossBanks(FitResults, PeakList, Variables=["PeakCentre", "Sigma"]):
    NumPeaks = len(PeakList)
    NumVars = len(Variables)

    # Create a gridspec object with the desired number of rows and columns
    #gs = gridspec.GridSpec(NumPeaks, NumVars * 2)
    fig, axs = plt.subplots(NumPeaks, NumVars)
    for Azimuth in [-17, -8.5, 0, 8.5, 17]:
        Angles = FitResults[np.isclose(FitResults["Azimuth"], Azimuth)]
        for PeakIndex in range(NumPeaks):
            for VarIndex in range(NumVars):
                #CurPlot=plt.subplot(gs[PeakIndex, VarIndex])
                VariableName = f"{PeakList.index[PeakIndex]}-{Variables[VarIndex]}"
                axs[PeakIndex, VarIndex].plot(Angles["TwoTheta"],Angles[VariableName]) 
                #CurPlot.plot(Angles["TwoTheta"],"Cu111-Sigma") #x="Sigma",
        
    
    
# fig, axes = plt.subplots(edgecolor='#ffffff', num='DiffnDataWS-dSp-grp-reb-Fit', subplot_kw={'projection': 'mantid'})
# axes.tick_params(axis='x', which='major', **{'gridOn': False, 'tick1On': True, 'tick2On': False, 'label1On': True, 'label2On': False})
# axes.tick_params(axis='y', which='major', **{'gridOn': False, 'tick1On': True, 'tick2On': False, 'label1On': True, 'label2On': False})
# axes.set_xlabel('axis-1')
# axes.set_ylabel('Integrated Intensity')
# legend = axes.legend().set_draggable(True).legend

# plt.show()
# # Scripting Plots in Mantid:
# # https://docs.mantidproject.org/tutorials/python_in_mantid/plotting/02_scripting_plots.html
#GroupingFile = os.path.join(r'C:/MantidInstall69/scripts/Engineering/calib', 'ENGINX_Texture30_grouping.xml') # ENGINX_5x24.xml ENGINX_Texture30_grouping.xml
GroupingFile = os.path.join(r'C:\MantidInstall\scripts\Engineering\calib', 'ENGINX_Texture30_grouping.xml')

df, nm =PeakAcrossBanks(346541, peak_list=[2.0881386024697948, 1.809843123112585, 1.276197610281563, 1.0906672907100898,
                                          1.0437930226911487, 0.9044813869705196, 0.8294169997234041, 0.8294169997234041,
                                          0.7379630082738616, 0.6953500373475515, 0.6114352638311255], PixelGrouping=GroupingFile, RebinParams=-0.0005)   #193749
 
#df.plot(x="TwoTheta",y="Cu111-Sigma") #x="Sigma",
#df.sort_values(by="Integrated Intensity")["Integrated Intensity"].reindex()
df
#df.sort_values(by="Integrated Intensity", inplace=True)
#df["Intensity rank"] = range(240)
# #df["Integrated Intensity"].plot(x="Intensity rank")
#df.plot(y="Integrated Intensity",x="Intensity rank")
#FitResults = df
#PlotPeaksAcrossBanks(FitResults, PeakList[:"F310"])
#FitResults[np.isclose(FitResults["Azimuth"], 0)]

In [ ]:
from mantid.simpleapi import *

def MakeGroupingFile(HorizDivs, OutputWkspName='GrpWksp', OutputFilename=None):
    CreateGroupingWorkspace(InstrumentName='ENGINX',OutputWorkspace=OutputWkspName)
    NumGroups = 10 * HorizDivs # 10 for 2*5 rows of detector modules on ENGIN-X
    PixelsInGroup = int(2400 / NumGroups)
    GroupList = []
    for GroupNo in range(1,NumGroups+1):
        GroupList = GroupList + [GroupNo] * PixelsInGroup
    print(GroupList)
    for i in range(len(GroupList)):
        mtd[OutputWkspName].setY(i, [GroupList[i]])
    if OutputFilename:
        SaveDetectorsGrouping(InputWorkspace=OutputWkspName, OutputFile=OutputFilename)
        
MakeGroupingFile(24, OutputFilename=os.path.join(GroupingFiles, "ENGINX_5x24.xml"))


In [ ]:
df.columns

In [ ]:
%matplotlib inline

#from scipy.stats import pearsonr
import scipy.stats as ss
from matplotlib.table import table

def AnalyseOrientationRelations(FitResDF,name,corr="Pearson"):
    """Look for orientation relationships in dataframe of peak fit results"""
    currdir = os.getcwd()
    savedir = os.path.join(currdir, "Saves")
    figName = f'{name}_Orientation_{corr}_Correlation.png'
    filedir = os.path.join(savedir, str(name))
    figdir = os.path.join(filedir, figName)
    Peaks = list({ColNamePart[0] for ColNamePart in df.columns.str.split("-") if len(ColNamePart) == 2}) # PeakList.index.to_numpy() #This retrieves the peaks
    NumPeaks = len(Peaks) #How many peaks there are
    Correlations = np.zeros((NumPeaks, NumPeaks)) #Matrix of size
    for PeakInd1 in range(NumPeaks):
        for PeakInd2 in range(NumPeaks):
            if corr == "Pearson":
                #The scipy and numpy pearson coefficients are interchangable, but the scipy one is easier to extract
                Correlations[PeakInd1,PeakInd2] = ss.pearsonr(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0]
                #print(np.corrcoef(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0][1])
                #Correlations[PeakInd1,PeakInd2] = np.corrcoef(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0][1]
            elif corr == "Spearman":
                Correlations[PeakInd1,PeakInd2] = ss.spearmanr(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0]
                print(ss.spearmanr(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity']))
            elif corr == "Kendall":
                Correlations[PeakInd1,PeakInd2] = ss.kendalltau(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0]
                #print(ss.kendalltau(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity']))
            elif corr == "Regression":
                Correlations[PeakInd1,PeakInd2] = ss.linregress(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity']).rvalue
                #print(ss.linregress(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity']))
            elif corr == "Xi":
                #Need to update to current version of scipy for this function to work. Recent coefficient for correlation which works on more complicated functions,
                #and may thus be more accurate.
                #https://souravchatterjee.su.domains/beam-correlation-trans.pdf
                #print(ss.chatterjeexi(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'],nan_policy='propagate'))
                Correlations[PeakInd1,PeakInd2] = ss.chatterjeexi(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'],nan_policy='propagate')[0]
            elif corr == "Weighted Kendall":
                #print(ss.weightedtau(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity']))
                Correlations[PeakInd1,PeakInd2] = ss.weightedtau(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0]
            else:
                Correlations[PeakInd1,PeakInd2] = ss.pearsonr(FitResDF[f'{Peaks[PeakInd1]}-Integrated Intensity'],FitResDF[f'{Peaks[PeakInd2]}-Integrated Intensity'])[0]
    fig, axes = plt.subplots(1,1)
    axes.axis('off')
    CorrelationsNormalised = (Correlations + 1) / 2.
    CellColours = plt.cm.RdYlGn(CorrelationsNormalised)
    CorrTable = plt.table(cellText=Correlations.round(decimals=3), cellColours=CellColours, loc='center', cellLoc='center',colLabels=Peaks, rowLabels=Peaks) # ,edges='open'
    CorrTable.set_fontsize(20)
    sortedCorr = sortCorrelations(Correlations,Peaks,NumPeaks)
    saveOrientation(name,corr,Correlations,NumPeaks,sortedCorr,Peaks)
    plt.savefig(figdir)
    #return Correlations
    #print(dir(CorrTable))
    #return Peaks

def sortCorrelations(Correlations,peaks,peaklen):
    #print(peaks)
    sortedDic = {}
    i = 0
    for peak in peaks:
        peakCorr = []
        #pOName = f'peak_{i}_ordered'
        orderedPeaks = []
        for j in range(peaklen):
            peakCorr.append(Correlations[i,j])
        peakCorrSorted = np.sort(peakCorr)
        peakCorrSorted = np.flip(peakCorrSorted)
        for sortedPeak in peakCorrSorted:
            orderedPeaks.append(peaks[int(np.argwhere(peakCorr == sortedPeak))])
        sortedDic[f'{peak}_ordered'] = orderedPeaks
        i = i + 1
    return sortedDic
            
#For some reason these runs aren't providing peaks
#Why these runs specifically - ask Joe
#These aren't cubic and so can't be fitted right now
df,nm = PeakAcrossBanks(216884, PixelGrouping=GroupingFile, RebinParams=-0.0001)
AnalyseOrientationRelations(df,nm,corr="Pearson")
df,nm = PeakAcrossBanks(216885, PixelGrouping=GroupingFile, RebinParams=-0.0001)
AnalyseOrientationRelations(df,nm,corr="Pearson")
df,nm = PeakAcrossBanks(216886, PixelGrouping=GroupingFile, RebinParams=-0.0001)
AnalyseOrientationRelations(df,nm,corr="Pearson")

#df,nm = PeakAcrossBanks(351364, PixelGrouping=GroupingFile, RebinParams=-0.0001)
#print(df)
#AnalyseOrientationRelations(df,nm,corr="Pearson")
#df,nm = PeakAcrossBanks(351464, PixelGrouping=GroupingFile, RebinParams=-0.0001)
#AnalyseOrientationRelations(df,nm,corr="Pearson")
#df,nm = PeakAcrossBanks(351538, PixelGrouping=GroupingFile, RebinParams=-0.0001)
#AnalyseOrientationRelations(df,nm,corr="Pearson")

In [ ]:
#Remaking this cell with lmfit instead of curve_fit
import math as m

def AnalyseGrainSize(FitResDF, name, title=""):
    Peaks = list({ColNamePart[0] for ColNamePart in df.columns.str.split("-") if len(ColNamePart) == 2}) # PeakList.index.to_numpy()
    NumPeaks = len(Peaks)
    fig, axes = plt.subplots(1,1)
    Sizes = {}
    grain_grads = {}
    for PeakInd in range(NumPeaks):
        IntensitySeries = f'{Peaks[PeakInd]}-Integrated Intensity'
        FitResDF[IntensitySeries] = FitResDF[IntensitySeries].mask(~FitResDF[IntensitySeries].between(1e-4,1e+4))
    #FitResDF.dropna(inplace=True)
    currdir = os.getcwd()
    savedir = os.path.join(currdir, "Saves")
    figName = f'{title}_GrainSize_graph.png'
    filedir = os.path.join(savedir, str(name))
    figdir = os.path.join(filedir, figName)
    for PeakInd in range(NumPeaks):
        Intensities = FitResDF[f'{Peaks[PeakInd]}-Integrated Intensity'].dropna().to_numpy()
        #print(Intensities)
        Intensities.sort()
        Intensities = Intensities[-2:2:-1]
        IntensityIndexes = np.array(range(len(Intensities)))
        plt.yscale('log')
        plt.ylim(1e-3, 3)
        plt.title(f'{title}')
        #plt.plot(Intensities)
        if len(Intensities) >= 2:
            #print(IntensityIndexes,Intensities)
            #How are these decided?
            a_const = 10
            grain_param = 1
            pfitNE = lm.create_params(a_psi = a_const, psi = grain_param)
            miniNE = lm.Minimizer(residualNE, pfitNE, fcn_args=(IntensityIndexes,Intensities),nan_policy='propagate')
            outNE = miniNE.leastsq()
            pfitCP = lm.create_params(a_alpha = a_const, alpha = grain_param)
            miniCP = lm.Minimizer(residualCP, pfitCP, fcn_args=(IntensityIndexes,Intensities),nan_policy='propagate')
            outCP = miniCP.leastsq()
            pfitWL = lm.create_params(a_w = a_const, w = grain_param)
            miniWL = lm.Minimizer(residualWL, pfitWL, fcn_args=(IntensityIndexes,Intensities),nan_policy='propagate')
            outWL = miniWL.leastsq()
            nameComp = ["NE","CP","WL"]
            fitComp = [abs(np.sum(outNE.residual)), abs(np.sum(outCP.residual)), abs(np.sum(outWL.residual))]
            bestOC = np.amin(fitComp)
            #print(bestOC, np.argwhere(fitComp == bestOC))
            fit = nameComp[np.amin(np.argwhere(fitComp == bestOC))]
            #print(fitComp, fit)
            if fit == "NE":
                best_fit = Intensities + outNE.residual
                psiFin = outNE.params['psi'].value
                plt.plot(Intensities, label=f'{PeakInd}--{round(psiFin,2)}({fit})')
                plt.plot(best_fit, color='black', linestyle='dashed')
                #Need to figure out coords more before using annotate
                #plt.annotate(text=f'{round(psiFin,2)}',xy=(IntensityIndexes[0],best_fit[0]))
                plt.legend()
                Sizes[Peaks[PeakInd]] = outNE.params['psi'].value # Second parameter is gradient-like parameter that relates to the sizes. Might be changed by lmfit
            elif fit == "CP":
                aFin = outCP.params['alpha'].value
                best_fit = Intensities + outCP.residual
                plt.plot(Intensities, label=f'{PeakInd}--{round(aFin,2)}({fit})')
                plt.plot(best_fit, color='black', linestyle='dashed')
                #plt.annotate(text=f'{round(aFin,2)}',xy=(IntensityIndexes[0],best_fit[0]))
                plt.legend()
                Sizes[Peaks[PeakInd]] = outCP.params['alpha'].value
            elif fit == "WL":
                wFin = outWL.params['w'].value
                best_fit = Intensities + outWL.residual
                plt.plot(Intensities, label=f'{PeakInd}--{round(wFin,2)}({fit})')
                plt.plot(best_fit, color='black', linestyle='dashed')
                #plt.annotate(text=f'{round(wFin,2)}',xy=(IntensityIndexes[0],best_fit[0]))
                plt.legend()
                Sizes[Peaks[PeakInd]] = outWL.params['w'].value
        else:
            Sizes[Peaks[PeakInd]] = None
    plt.savefig(figdir)
    return Sizes
    
def NegativeExp(x, a_psi, psi):
    return a_psi*np.exp(-(x/psi))
def residualNE(pars,x,data):
    model = NegativeExp(x, a_psi=pars['a_psi'],psi=pars['psi']) 
    return model - data
    
def ConstPower(x, a_alpha, alpha):
    #return a_alpha*(x**(-float(alpha)))
    #Temporary fix. This function specifically causes a lot of issue with nans
    return a_alpha*(x**(int(alpha)))
def residualCP(pars,x,data):
    model = ConstPower(x, a_alpha=pars['a_alpha'],alpha=pars['alpha']) 
    return model - data

def WLorentz(x, a_w, w):
    return a_w*(w**2/(w**2+x**2))
def residualWL(pars,x,data):
    model = WLorentz(x, a_w=pars['a_w'],w=pars['w']) 
    return model - data


#df.plot(x="TwoTheta",y="PeakCentre") #x="Sigma",
#df.sort_values(by="Integrated Intensity")["Integrated Intensity"].reindex()
#df
# df.sort_values(by="Integrated Intensity", inplace=True)
# df["Intensity rank"] = range(240)
# #df["Integrated Intensity"].plot(x="Intensity rank")

#GroupingFile = os.path.join(r'C:/MantidInstall69/scripts/Engineering/calib', 'ENGINX_Texture30_grouping.xml') # ENGINX_5x24.xml ENGINX_Texture30_grouping.xml
GroupingFile = os.path.join(r'C:\MantidInstall\scripts\Engineering\calib', 'ENGINX_Texture30_grouping.xml')

sizes = []
#UniquePointRange = list(range(324884,324934))
#for duplicate in [324897,324899,324905,324907,324913,324915]:
#    UniquePointRange.remove(duplicate)
#print(UniquePointRange)
UniquePointRange = list(range(351464,351470))
for RunNo in UniquePointRange: # range(324884,324975):   # from 324884
    df,nm = PeakAcrossBanks(RunNo, PixelGrouping=GroupingFile, RebinParams=-0.0005)
    grainsize = AnalyseGrainSize(df, nm, title=RunNo)
    grainsize["X"]=mtd["DiffnDataWS"].getRun()["x_position"].timeAverageValue()
    grainsize["Y"]=mtd["DiffnDataWS"].getRun()["y_position"].timeAverageValue()
    grainsize["Z"]=mtd["DiffnDataWS"].getRun()["z_position"].timeAverageValue()
    grainsizedf = pd.DataFrame(grainsize, index=[RunNo])
    sizes.append(grainsizedf)

SizeDF = pd.concat(sizes)
#fig, axes = plt.subplots(1,1)
#plt.plot(SizeDF)
SizeDF
# df.plot(y="Integrated Intensity",x="Intensity rank")
#df


In [ ]:
print(SizeDF.keys())
SizeDF = SizeDF.fillna(0)
print(SizeDF)
#AnalyseGrainSize(df,nm)

In [ ]:
'''
Contour plotter using XYZ data in Excel

Joe Kelleher 2010

'''
%matplotlib ipympl
import win32com.client
from numpy import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches


XSteps = 200
YSteps = 200
#XSteps = 10
#YSteps = 10

#MinValue = None #-250 # Can be set to None for auto scaling
#MaxValue = None #250
#ContourLevels = arange(MinValue, MaxValue+1, 50)

LengthScalesOnImage = False

XAxisFontSize = 16
YAxisFontSize = 16
ColourBarFontSize = 16
# Colour bar position and size changed by numbers below
ColourBar_BottomLeft_X = 0.95
ColourBar_BottomLeft_Y = 0.2
ColourBar_Width = 0.05
ColourBar_Height = 0.6

'''
def PlotXYZfromXL():
    XL = GetExcelWorkbook("SSPL data")
    XYZData = ReadExcelList(XL, "SSPL data", "A4:C4", ExtendDown=True)
    ShapeData = ReadExcelList(XL, "Shape", "A2:B2", ExtendDown=True)
    print(XYZData)
    #XValues, YValues, ZTable, FValues = BuildTableFromList(XYZData[0], XYZData[1], XYZData[2])
    #XYF = XYZData[0:3]
    #XValues, YValues, FValues = XYF
    XValues, YValues, ZValues = XYZData[:,0], XYZData[:,1], XYZData[:,2]
    XYF = FindF(XValues, YValues, ZValues)
    FValues = XYF[2]
    XGridValues = arange(min(ShapeData[:,0]), max(ShapeData[:,0]), (max(ShapeData[:,0]) - min(ShapeData[:,0])) / XSteps)
    YGridValues = arange(min(ShapeData[:,1]), max(ShapeData[:,1]), (max(ShapeData[:,1]) - min(ShapeData[:,1])) / YSteps)
    print(XGridValues, YGridValues)
    #XGrid, YGrid = meshgrid(XGridValues, YGridValues)
    #print XGrid.ravel()
    #print YGrid.ravel()
    #ZGrid = EvaluateZWithVectors(XGrid.ravel(), YGrid.ravel(), XYF)
    #print type(ZGrid)
    # Evaluate row-at-a-time due to memory constraints
    ZGrid = array([[]]* len(XGridValues))
    for YGridValue in YGridValues[::1]:
        ZGridForY = EvaluateZ(XGridValues, array([YGridValue] * len(XGridValues)), XYF)
        ZGrid = concatenate((ZGrid, ZGridForY.reshape((-1, 1))), axis=1)
    MakeContourPlot(XGridValues, YGridValues, ZGrid.transpose(), Outline=ShapeData, MarkerPoints=XYF[0:2])
    #MakeContourPlot(XGridValues, YGridValues, ZGrid, Outline=ShapeData, MarkerPoints=XYF[0:2])
    #patch = matplotlib.patches.Circle((10,10), radius=100)
    #im.set_clip_path(patch)
'''

def MakeContourPlot(XGridValues, YGridValues, ZGrid, ContourLevels, Outline=None, MarkerPoints=None):
    fig = plt.figure()
    if LengthScalesOnImage:
       ax = fig.add_subplot(111, frameon=False) # , xticks=[], yticks=[]
    else:
        ax = fig.add_subplot(111, frameon=False, xticks=[], yticks=[])
    ContourLines = ax.contour(XGridValues, YGridValues, ZGrid, colors = 'k', levels=ContourLevels)
    #for c in ContourLines.collections:
    #    c.set_linestyle('solid') # Don't use dashed line for <0 contours
    ImageExtent = (XGridValues[0], XGridValues[-1], YGridValues[0], YGridValues[-1])
    BackgroundImage = ax.imshow(ZGrid, extent=ImageExtent, origin='lower', vmin=min(ContourLevels), vmax=max(ContourLevels))
    if Outline != None:
        #OutlinePath = path.Path(vertices=tuple(Outline))
        OutlinePath = patches.Polygon(Outline, closed=True, facecolor='none')
        ax.add_patch(OutlinePath)
        BackgroundImage.set_clip_path(OutlinePath)
        ax.set_clip_path(OutlinePath)
        #ax.set_clip_on(True)
        #for c in ContourLines.collections:
        #    c.set_clip_path(OutlinePath)
        #    c.set_clip_on(True)
    #ax.set_xticks(np.linspace(0,len(XValues),num=10,dtype=float), labels=np.around(np.linspace(XValues[0],XValues[-1],num=10,dtype=float),decimals=2))
    #ax.set_yticks(np.linspace(0,len(YValues),num=10,dtype=float), labels=np.around(np.linspace(YValues[0],YValues[-1],num=10,dtype=float),decimals=2))
    ColourBarAxes = fig.add_axes([ColourBar_BottomLeft_X, ColourBar_BottomLeft_Y, ColourBar_Width, ColourBar_Height])
    ColorBar = fig.colorbar(BackgroundImage, cax=ColourBarAxes)
    if MarkerPoints != None:
        MarkerPoints = ax.scatter(reshape(MarkerPoints[0],-1),reshape(MarkerPoints[1],-1),marker='x',c='k',s=0.05,zorder=10)
    for label in ax.xaxis.get_ticklabels():
        # label is a Text instance
        label.set_fontsize(XAxisFontSize)
    for label in ax.yaxis.get_ticklabels():
        # label is a Text instance
        label.set_fontsize(YAxisFontSize)
    for label in ColorBar.ax.yaxis.get_ticklabels():
        # label is a Text instance
        label.set_fontsize(ColourBarFontSize)

    plt.show()

def FFromZMatrix(KnownX, KnownY):
    KnownXRow = reshape(KnownX, -1)
    KnownYRow = reshape(KnownY, -1)
    DiffX = KnownXRow[:,newaxis] - KnownXRow[:]
    DiffY = KnownYRow[:,newaxis] - KnownYRow[:]
    r2 = (DiffX ** 2) + (DiffY ** 2)
    RadialBasis = r2 * log(r2 + 0.0000001)
    PlaneFit = concatenate(([ones(KnownXRow.shape[0])], [KnownXRow], [KnownYRow]), 0)   # Three rows at bottom
    FFromZ = concatenate((concatenate((RadialBasis, PlaneFit), 0), 
                       concatenate((transpose(PlaneFit), zeros([3, 3])), 0)), 1)
    return FFromZ


def FindF(KnownX, KnownY, KnownZ):
    KnownXRow = reshape(KnownX, -1)
    KnownYRow = reshape(KnownY, -1)
    KnownZRow = concatenate((reshape(KnownZ, -1), zeros(3)), 0)
    SolutionMatrix = FFromZMatrix(KnownXRow, KnownYRow)
    F = reshape(linalg.lstsq(SolutionMatrix, KnownZRow)[0], -1)
    return [KnownXRow, KnownYRow, F]

def EvaluateZ(SoughtX, SoughtY, XYF):
    SoughtXCol = reshape(SoughtX, (-1, 1))
    SoughtYCol = reshape(SoughtY, (-1, 1))
    ZFromF = concatenate((
                ((SoughtXCol - XYF[0]) ** 2 + (SoughtYCol - XYF[1]) ** 2) *
                   log((SoughtXCol - XYF[0]) ** 2 + (SoughtYCol - XYF[1]) ** 2 + 0.0000001),
                ones((SoughtXCol.shape[0], 1)),
                SoughtXCol,
                SoughtYCol), 1)
    ZValues = dot(ZFromF, XYF[2])
    return ZValues

def EvaluateZWithVectors(SoughtX, SoughtY, XYF):
    SoughtXCol = reshape(SoughtX, (-1, 1))
    SoughtYCol = reshape(SoughtY, (-1, 1))
    
    #SoughtXYArray = array([SoughtXCol, SoughtYCol])
    #KnownXYArray = array([[XYF[0]], [XYF[1]]])
    XDifferencesSq = (SoughtXCol - XYF[0]) ** 2
    YDifferencesSq = (SoughtYCol - XYF[1]) ** 2
    #Test = Test ** 2
    RSquared = XDifferencesSq + YDifferencesSq #sum((SoughtXYArray - KnownXYArray)**2,1)
    RSqLnRSq = RSquared * log(RSquared + 0.0000001) # Epsilon
    ZValues = XYF[2][-3] + XYF[2][-2] * SoughtXCol + XYF[2][-1] * SoughtYCol \
        + sum(XYF[2][:-3] * RSqLnRSq)
    #StressTensor = StressData[-3,3:8:2] + XPosition * StressData[-2,3:8:2] \
    #                                    + YPosition * StressData[-1,3:8:2] \
    #               + sum(StressData[0:-3,3:8:2] * transpose(array([RSqLnRSq,RSqLnRSq,RSqLnRSq])))
    #    ZFromF = concatenate((
    #                ((SoughtXCol - XYF[0]) ** 2 + (SoughtYCol - XYF[1]) ** 2) *
    #                   log((SoughtXCol - XYF[0]) ** 2 + (SoughtYCol - XYF[1]) ** 2 + 0.0000001),
    #                ones((SoughtXCol.shape[0], 1)),
    #                SoughtXCol,
    #                SoughtYCol), 1)
    #    ZValues = matrixmultiply(ZFromF, XYF[2])
    return ZValues

def StressAtPoint(self, XPosition, YPosition, StressData):
    #Epsilon = 0.0000001
    # First work out StressX, StressY, StressShear
    # StressTensor is [ StressX, StressY, StressShear ]
    #StressX = StressData[-3,3] + XPosition * StressData[-2,3] \
    #                                        + YPosition * StressData[-1,3]
    #StressY = StressData[-3,5] + XPosition * StressData[-2,5] \
    #                                        + YPosition * StressData[-1,5]
    #StressShear = StressData[-3,7] + XPosition * StressData[-2,7] \
    #                                            + YPosition * StressData[-1,7]
    #RSquared = (XPosition - StressData[0:-3,0]) ** 2 + (YPosition - StressData[0:-3,1]) ** 2
    #RSqLnRSq = RSquared * log(RSquared + 0.0000001) # Epsilon
    RSquared = sum(((XPosition, YPosition) - StressData[0:-3,0:2])**2,1)
    RSqLnRSq = RSquared * log(RSquared + 0.0000001) # Epsilon
    StressTensor = StressData[-3,3:8:2] + XPosition * StressData[-2,3:8:2] \
                                        + YPosition * StressData[-1,3:8:2] \
                   + sum(StressData[0:-3,3:8:2] * transpose(array([RSqLnRSq,RSqLnRSq,RSqLnRSq])))
    # StressTensor = StressTensor + ...
    #StressX = StressX + Numeric.sum(StressData[0:-3,3] * RSqLnRSq)
    #StressY = StressY + Numeric.sum(StressData[0:-3,5] * RSqLnRSq)
    #StressShear = StressShear + Numeric.sum(StressData[0:-3,7] * RSqLnRSq)
    return StressTensor


def BuildTableFromList(XValues, YValues, ZValues=None, FValues=None):
    if FValues == None:
        XYF = FindF(XValues, YValues, ZValues)
        FValues = XYF[2]
    XRange = max(XValues) - min(XValues)
    YRange = max(YValues) - min(YValues)
    XGridValues = arange(min(XValues), max(XValues), XRange / XSteps)
    YGridValues = arange(min(YValues), max(YValues), YRange / YSteps)
    XGrid, YGrid = meshgrid(XGridValues, YGridValues)
    print(XGrid.ravel())
    print(YGrid.ravel())
    ZGrid = EvaluateZ(XGrid.ravel(), YGrid.ravel(), XYF) #WithVectors
    ZGrid = ZGrid.reshape((len(XGridValues), len(YGridValues)))
    #print(ZGrid.ravel())
    return XGridValues, YGridValues, ZGrid, XYF

def IsInShape(self, XPosition, YPosition):
    #pywin.debugger.brk()
    NumberOfPoints = len(self.EdgePoints)
    EdgesToLeft = 0
    CurrentPoint = 0
    Y2 = self.EdgePoints[-1][1]
    while CurrentPoint < NumberOfPoints:
        # Get Y positions, Y1 and Y2 of two consecutive points in the list
        Y1 = Y2
        Y2 = self.EdgePoints[CurrentPoint][1]
        # If requested Y value lies between them, work out X position on this edge with
        # the same Y value as the requested Y value
        if (Y2 >= YPosition) == (YPosition > Y1):
            X1 = self.EdgePoints[CurrentPoint - 1][0]  # Final value when CurrentPoint = 0
            X2 = self.EdgePoints[CurrentPoint][0]
            EdgeXPosition = X1 + ((X2 - X1) * ((YPosition - Y1) / (Y2 - Y1)))
            if EdgeXPosition <= XPosition:
                EdgesToLeft = EdgesToLeft + 1
        CurrentPoint = CurrentPoint + 1
    # Final answer depends on whether an even or odd number of edges were crossed
    if EdgesToLeft % 2 == 1:
        return True
    else:
        return False

'''
def ReadExcelList(XLWorkbook, XLWorksheetName, XLCellRange, ExtendDown=True):
    RangeTop = XLWorkbook.Sheets(XLWorksheetName).Range(XLCellRange)
    ColumnCount = RangeTop.Columns.Count
    print(ColumnCount, " columns")
    ExtraRows = 0
    if ExtendDown:
        while RangeTop.Cells(101 + ExtraRows,1).Value != None:
            ExtraRows = ExtraRows + 100
        while RangeTop.Cells(11 + ExtraRows,1).Value != None:
            ExtraRows = ExtraRows + 10
        while RangeTop.Cells(1 + ExtraRows,1).Value != None:
            ExtraRows = ExtraRows + 1
    RangeCells = XLWorkbook.Sheets(XLWorksheetName).Range(RangeTop.Cells(1,1),
                                                              RangeTop.Cells(ExtraRows, ColumnCount)).Value
    return array(RangeCells)

def GetExcelWorkbook(WorksheetName):
    "Get a COM reference to a workbook with a given worksheet name"
    XL = win32com.client.Dispatch("Excel.Application")
    XL.Visible = 1
    XLBooksCount = XL.Workbooks.Count
    FoundBook = False
    for XLBookNo in range(1, XLBooksCount + 1):
        try:
            XLSheet = XL.Workbooks(XLBookNo).Sheets(WorksheetName)
            # If that didn't generate exception, we've found a workbook with the sheet in it
            FoundBook = True
            print("Found already open file...")
        except:
            pass
        if FoundBook == True:
            break
    XLBook = XL.Workbooks(XLBookNo)
    print(XLBook.Name)
    XL.ScreenUpdating = True
    return XLBook

#if __name__ == '__main__':
#    PlotXYZfromXL()
'''
54